In [ ]:
import pandas as pd
import numpy as np 
from scipy import sparse

In [ ]:
def abrir_e_criar_lista(diretorio, separador, coluna):
    variavel = open(diretorio, 'r', encoding='iso-8859-1')

    lista = []

    for linha in variavel:
        linha = linha.split(separador)
        lista.append(linha[coluna]) # adicionar na lista a coluna que eu escolher

    return lista

In [ ]:
userID_data = abrir_e_criar_lista('/home/laryssacosta/workspace/python-warmup/ml-100k/u.data', '\t', 0) # coluna 0 é o userID
movieID_data = abrir_e_criar_lista('/home/laryssacosta/workspace/python-warmup/ml-100k/u.data', '\t', 1)
ratings_data = abrir_e_criar_lista('/home/laryssacosta/workspace/python-warmup/ml-100k/u.data', '\t', 2)

In [ ]:
array_data = np.array([userID_data, movieID_data, ratings_data])

In [ ]:
array_data.shape

In [ ]:
df_data = pd.DataFrame(data=array_data) # ver dados com pandas
df_data = df_data.T # para transpor
df_data.columns = 'userID movieID rating'.split() # mudar nomes das colunas
df_data

In [ ]:
movieID_unicos = abrir_e_criar_lista('/home/laryssacosta/workspace/python-warmup/ml-100k/u.item', '|', 0)
titulo_movie = abrir_e_criar_lista('/home/laryssacosta/workspace/python-warmup/ml-100k/u.item', '|', 1)
titulo_movie[:5]

In [ ]:
array_item = np.array([movieID_unicos, titulo_movie]) # lista com ID e nome dos filmes
array_item

In [ ]:
df_item = pd.DataFrame(data=array_item)
df_item = df_item.T
df_item.columns = 'movieID titulo'.split() # renomear colunas - tem que ser o mesmo titulo para poder dar o merge
df_item

In [ ]:
df_data_item = pd.merge(df_data, df_item) # para juntar os dados e criar uma nova coluna com o titulo
df_data_item

In [ ]:
df_data_item['rating'] = pd.to_numeric(df_data_item['rating'])

In [ ]:
matriz_user_movie = df_data_item.pivot_table(values='rating', columns='titulo', index='userID') # criando a matriz

In [ ]:
matriz_user_movie 

In [ ]:
#substituir NaN por 0
matriz_user_movie = matriz_user_movie.dropna(thresh=20, axis=1) # ter pelo menos 20 avaliações

In [ ]:
matriz_user_movie.head()

In [ ]:
matriz_user_movie = matriz_user_movie.fillna(0) # onde tem NaN, colocar 0
matriz_user_movie.head()

In [ ]:
#zscore
# todos os usuarios estao no mesmo nível
def padronizacao(linha):
    nova_linha = (linha - linha.mean())/(linha.max() - linha.min())
    return nova_linha

In [ ]:
matriz_user_movie_std = matriz_user_movie.apply(padronizacao) # aplicar a funçao que criamos

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

## Filtro colaborativo --> Item Item --> User User

- primeiro item item

In [ ]:
item_similarity_cos_array = cosine_similarity(matriz_user_movie_std.T) # matriz padronizada transposta

In [ ]:
pd.DataFrame(item_similarity_cos_array, columns = matriz_user_movie.columns, index = matriz_user_movie.columns) # linhas e colunas tem que ser os mesmos para comparar filme com filme

- user user

In [ ]:
user_similarity_cos_array = cosine_similarity(matriz_user_movie_std)

In [ ]:
user_similarity_cos = pd.DataFrame(data = user_similarity_cos_array, columns= matriz_user_movie_std.index, index = matriz_user_movie_std.index)

In [ ]:
user_similarity_cos

- Correlação de Pearson

In [ ]:
item_similarity_pearson = matriz_user_movie.corr(method='pearson')

In [ ]:
user_similarity_pearson = matriz_user_movie.T
user_similarity_pearson = user_similarity_pearson.corr(method='pearson')

In [ ]:
def get_similar_user(user_ID, rating):

    similar = user_similarity_pearson[user_ID]*rating
    similar = similar.sort_values(ascending=False)

    return similar.head()


In [ ]:
user_1 = get_similar_user('1', 2)
user_1